In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('./course-data/Advertising.csv')

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
X = df.drop('sales',axis=1)

In [5]:
y = df['sales']

---

In [6]:
from sklearn.preprocessing import PolynomialFeatures

In [7]:
pol_trafo = PolynomialFeatures(degree=3, include_bias=False)

In [10]:
pol_trafo.fit(X)

PolynomialFeatures(degree=3, include_bias=False)

In [11]:
pol_trafo.get_feature_names(X.columns)

['TV',
 'radio',
 'newspaper',
 'TV^2',
 'TV radio',
 'TV newspaper',
 'radio^2',
 'radio newspaper',
 'newspaper^2',
 'TV^3',
 'TV^2 radio',
 'TV^2 newspaper',
 'TV radio^2',
 'TV radio newspaper',
 'TV newspaper^2',
 'radio^3',
 'radio^2 newspaper',
 'radio newspaper^2',
 'newspaper^3']

In [12]:
pol_features = pol_trafo.transform(X)

In [23]:
# ndarray of poly trafo features
pol_features.shape

(200, 19)

In [18]:
# dataframe of poly trafo features
X_pol_trafo = pd.DataFrame(pol_features, columns=pol_trafo.get_feature_names(X.columns))

In [19]:
X_pol_trafo.head()

,TV,radio,newspaper,TV^2,TV radio,TV newspaper,radio^2,radio newspaper,newspaper^2,TV^3,TV^2 radio,TV^2 newspaper,TV radio^2,TV radio newspaper,TV newspaper^2,radio^3,radio^2 newspaper,radio newspaper^2,newspaper^3
0,230.1,37.8,69.2,52946.01,8697.78,15922.92,1428.84,2615.76,4788.64,1.218288e+07,2001359.178,3663863.892,328776.084,601886.376,1101866.064,54010.152,98875.728,181010.592,331373.888
1,44.5,39.3,45.1,1980.25,1748.85,2006.95,1544.49,1772.43,2034.01,8.812112e+04,77823.825,89309.275,68729.805,78873.135,90513.445,60698.457,69656.499,79936.593,91733.851
2,17.2,45.9,69.3,295.84,789.48,1191.96,2106.81,3180.87,4802.49,5.088448e+03,13579.056,20501.712,36237.132,54710.964,82602.828,96702.579,146001.933,220434.291,332812.557
3,151.5,41.3,58.5,22952.25,6256.95,8862.75,1705.69,2416.05,3422.25,3.477266e+06,947927.925,1342706.625,258412.035,366031.575,518470.875,70444.997,99782.865,141338.925,200201.625
4,180.8,10.8,58.4,32688.64,1952.64,10558.72,116.64,630.72,3410.56,5.910106e+06,353037.312,1909016.576,21088.512,114034.176,616629.248,1259.712,6811.776,36834.048,199176.704


In [15]:
pol_features_df.shape

(200, 19)

---

In [16]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_pol_trafo, y, test_size=0.33, random_state=42)

---

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
scaler = StandardScaler()

In [27]:
# we have to fit the instance of the StandardScaler to compute the statistics; it is important to avoid data leakage (from the test set)!
scaler.fit(X_train)

StandardScaler()

In [28]:
X_train_scaled = scaler.transform(X_train)

In [29]:
X_test_scaled = scaler.transform(X_test)

In [34]:
for i, column in enumerate(X_pol_trafo):
    print(f'{column}: {X_train_scaled[0][i]}')

TV: 1.6675933595243773
radio: 0.26512404474449547
newspaper: -1.395536624584335
TV^2: 2.2015037251311553
TV radio: 1.2775702597215366
TV newspaper: -0.9050233092675088
radio^2: -0.021316020241327022
radio newspaper: -0.8758223006221503
newspaper^2: -0.8399005581595183
TV^3: 2.64499976035433
TV^2 radio: 1.8207324927559239
TV^2 newspaper: -0.6501089609340481
TV radio^2: 0.656672040208119
TV radio newspaper: -0.6414214631292395
TV newspaper^2: -0.6127684143457218
radio^3: -0.23015660766502286
radio^2 newspaper: -0.7133113005297929
radio newspaper^2: -0.6502876624686
newspaper^3: -0.5691177845412948


---

In [35]:
from sklearn.linear_model import Ridge

In [ ]:
# help(Ridge)

In [36]:
ridge_model = Ridge(alpha=10)

In [37]:
ridge_model.fit(X_train_scaled, y_train)

Ridge(alpha=10)

In [39]:
ridge_test_pred = ridge_model.predict(X_test_scaled)

In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [43]:
MAE = mean_absolute_error(y_test, ridge_test_pred)

In [44]:
MAE

0.6329556348463364

In [45]:
RMSE = np.sqrt(mean_squared_error(y_test, ridge_test_pred))

In [46]:
RMSE

0.8927190051123458

---

In [47]:
from sklearn.linear_model import RidgeCV

In [48]:
# we use all the default values; alphas=(0.1, 1.0, 10.0)
ridge_cv_model = RidgeCV()

In [49]:
# we use the training set for puposes of hyper-parameter tuning. With CV the train set is split into train set and validation set. In tis case y_test becomes the hold out set
ridge_cv_model.fit(X_train_scaled,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))

In [50]:
# to determine what alpha is performing the best, the model is using the default scoring metric 
ridge_cv_model.alpha_

0.1

In [51]:
from sklearn.metrics import SCORERS

In [59]:
# SCORERS.keys()

In [56]:
ridge_cv_model = RidgeCV(scoring='neg_mean_absolute_error')

In [57]:
ridge_cv_model.fit(X_train_scaled,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [58]:
ridge_cv_model.alpha_

0.1

In [60]:
ridge_cv_test_pred = ridge_cv_model.predict(X_test_scaled)

In [61]:
MAE = mean_absolute_error(y_test, ridge_cv_test_pred)

In [62]:
# versus ridge_no_cv with 0.6329556348463364
MAE

0.43430757664842945

In [65]:
RMSE = np.sqrt(mean_squared_error(y_test, ridge_cv_test_pred))

In [66]:
# verus ridge_no_cv_RMSE = 0.8927190051123458
RMSE
# with an alpha value of 0.1 (which was found to be the best alpha value according to CV on the training set) and against the hold-out test (we do NOT tune based off this test resutl)

0.5635899169556882

In [67]:
ridge_cv_model.coef_

array([ 5.84681185,  0.52142086,  0.71689997, -6.17948738,  3.75034058,
       -1.36283352, -0.08571128,  0.08322815, -0.34893776,  2.16952446,
       -0.47840838,  0.68527348,  0.63080799, -0.5950065 ,  0.61661989,
       -0.31335495,  0.36499629,  0.03328145, -0.13652471])

In [94]:
feature_names = pol_trafo.get_feature_names(X.columns)

for i, feature in enumerate(feature_names):
    print(f'{feature} = {ridge_cv_model.coef_[i]:.4f}')
    
# note that none of this coefficients are zero (or close to zero)

TV = 5.8468
radio = 0.5214
newspaper = 0.7169
TV^2 = -6.1795
TV radio = 3.7503
TV newspaper = -1.3628
radio^2 = -0.0857
radio newspaper = 0.0832
newspaper^2 = -0.3489
TV^3 = 2.1695
TV^2 radio = -0.4784
TV^2 newspaper = 0.6853
TV radio^2 = 0.6308
TV radio newspaper = -0.5950
TV newspaper^2 = 0.6166
radio^3 = -0.3134
radio^2 newspaper = 0.3650
radio newspaper^2 = 0.0333
newspaper^3 = -0.1365


In [70]:
# ridge_cv_model.best_score_

---

In [73]:
from sklearn.linear_model import LassoCV

# there are two ways to determine the alpha hyperparameter: (a) provide list of alphas as an array (b) alpha can be set automatically by the class based off epsilon and n_alphas (we use the default values)

In [84]:
lasso_cv_model = LassoCV(max_iter=1000000)

# another solution would be lasso_cv_model = LassoCV(eps=0.1)

In [85]:
lasso_cv_model.fit(X_train_scaled, y_train)

# the warning (nor an error) means that the scochastic search for the alpha value did not converge -> quick fix: (a) increase the max. number of iterations in the model instantiation or (b) increase the epsilon search value; epsilon is the length of the path, i.e. the ratio alpha_min / alpha_max. There is also the possiblity to play with the tolerance level of the model. 

LassoCV(max_iter=1000000)

In [86]:
lasso_cv_model.alpha_

0.004968802520343366

In [88]:
lasso_cv_test_pred = lasso_cv_model.predict(X_test_scaled)

In [89]:
# versus MAE_ridge_cv = 0.43430757664842945

MAE = mean_absolute_error(y_test, lasso_cv_test_pred)
MAE

0.46291883026932984

In [91]:
# versus RMSE_ridge_cv = 0.5635899169556882

RMSE = np.sqrt(mean_squared_error(y_test, lasso_cv_test_pred))
RMSE

0.5785146895301977

In [95]:
feature_names = pol_trafo.get_feature_names(X.columns)

for i, feature in enumerate(feature_names):
    print(f'{feature} = {lasso_cv_model.coef_[i]:.4f}')
    
# note that allthough the performce is less than RidgeCV, many of the beta values are zero -> the model is less complex, we drop almost half of all those features and we get almost the exact same performance. Beta values are still hard to interpret. 
# we could also expand the search for the best possible Lasso by addint in more feature coefficients and getting a better performance model. Trade-off: complex model with high performance  versus simple model (easier to interpret) with less performance

TV = 5.1961
radio = 0.4304
newspaper = 0.2988
TV^2 = -4.8042
TV radio = 3.4667
TV newspaper = -0.4051
radio^2 = 0.0000
radio newspaper = 0.0000
newspaper^2 = 0.0000
TV^3 = 1.3526
TV^2 radio = -0.0000
TV^2 newspaper = 0.0000
TV radio^2 = 0.1488
TV radio newspaper = -0.0000
TV newspaper^2 = 0.0000
radio^3 = 0.0000
radio^2 newspaper = 0.0965
radio newspaper^2 = 0.0000
newspaper^3 = 0.0435


---

In [96]:
lasso_cv_model = LassoCV(eps=0.1)

In [97]:
lasso_cv_model.fit(X_train_scaled, y_train)

# the warning (nor an error) means that the scochastic search for the alpha value did not converge -> quick fix: (a) increase the max. number of iterations in the model instantiation or (b) increase the epsilon search value; epsilon is the length of the path, i.e. the ratio alpha_min / alpha_max. There is also the possiblity to play with the tolerance level of the model. 

LassoCV(eps=0.1)

In [98]:
lasso_cv_model.alpha_

0.4968802520343365

In [99]:
lasso_cv_test_pred = lasso_cv_model.predict(X_test_scaled)

In [100]:
# versus MAE_ridge_cv = 0.43430757664842945

MAE = mean_absolute_error(y_test, lasso_cv_test_pred)
MAE

0.6791873341672893

In [101]:
# versus RMSE_ridge_cv = 0.5635899169556882

RMSE = np.sqrt(mean_squared_error(y_test, lasso_cv_test_pred))
RMSE

1.0399114953979072

In [102]:
feature_names = pol_trafo.get_feature_names(X.columns)

for i, feature in enumerate(feature_names):
    print(f'{feature} = {lasso_cv_model.coef_[i]:.4f}')
    
# note that allthough the performce is less than RidgeCV and LassoCV with more iterations, the model is only considering two features -> models is less complex and we can easily interpret this model

TV = 0.9675
radio = 0.0000
newspaper = 0.0000
TV^2 = 0.0000
TV radio = 3.8428
TV newspaper = 0.0000
radio^2 = 0.0000
radio newspaper = 0.0000
newspaper^2 = 0.0000
TV^3 = 0.0000
TV^2 radio = 0.0000
TV^2 newspaper = 0.0000
TV radio^2 = 0.0000
TV radio newspaper = 0.0000
TV newspaper^2 = 0.0000
radio^3 = 0.0000
radio^2 newspaper = 0.0000
radio newspaper^2 = 0.0000
newspaper^3 = 0.0000


---

In [103]:
# In a start here! 

from sklearn.linear_model import ElasticNetCV

In [104]:
# note that l1_ratio refers to the alpha shows in the formula and alphas the lambda outside the beta terms

elastic_cv_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1],max_iter=1000000)

In [105]:
elastic_cv_model.fit(X_train_scaled, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [106]:
# the model disregards Ridge completely
elastic_cv_model.l1_ratio_

1.0

In [107]:
# note that this model is identical to the Lasso model with max_iter=1000000
elastic_cv_model.alpha_

0.004968802520343366